# Data preparation: convert idomaar to CSV

Convert [30Music dataset](http://crowdrec.eu/2015/11/30music-dataset-release/) from [idomaar format](https://github.com/crowdrec/idomaar/wiki/DATA-FORMAT) to CSV format.

In [4]:
import sys, csv, json
import numpy as np
import pandas as pd

In [5]:
COLUMNS = ['Type', 'ID', 'Timestamp', 'Properties', 'LinkedEntities']

## Convert albums data to CSV

In [ ]:
falbums = 'albums.idomaar'

In [ ]:
#with open(falbums) as tsvin:
#    tsvin = csv.reader(tsvin, delimiter='\t')
#    for row in tsvin:
#        if len(row[4]) > 2:
#            print(row)

In [ ]:
albums = pd.read_csv(falbums, delimiter='\t', header=None)

In [ ]:
albums.columns = COLUMNS

In [ ]:
albums.head()

In [ ]:
albums[albums['Timestamp'] != -1]

In [ ]:
albums[albums['LinkedEntities'] != '{}']

In [ ]:
albums.set_index('ID', inplace=True)

In [ ]:
albums.head()

Deal with illegally formated JSON.

In [ ]:
cnt = 0
for ix in albums.index:
    try:
        prop = json.loads(albums.loc[ix, 'Properties'])
        aa = (prop['MBID'], prop['title'])
    except:
        cnt += 1
        #print(ix)
print(cnt)

In [ ]:
def parse_properties(props, debug=False): 
    try:
        prop = json.loads(props)
    except:
        # deal with duplicated " in json string
        props = props.replace('"title":', '"TITLE":').replace('"', '').replace('\\', '')\
                     .replace('MBID:', '"MBID":"').replace(', TITLE:', '", "title":"').replace('}', '"}')
        if debug is True: print(props)
        prop = json.loads(props)
    return pd.Series({'MBID': prop['MBID'], 'Title': prop['title']})

In [ ]:
props = albums.loc[708, 'Properties']
props

In [ ]:
#json.loads(props)  # causes exception

In [ ]:
parse_properties(props)

In [ ]:
prop_df = albums['Properties'].apply(lambda s: parse_properties(s, debug=False))

In [ ]:
prop_df.head()

In [ ]:
albums = albums.merge(prop_df, left_index=True, right_index=True)

In [ ]:
albums.head()

In [ ]:
albums.drop(['Type', 'Timestamp', 'Properties', 'LinkedEntities'], axis=1, inplace=True)

In [ ]:
albums.head()

In [ ]:
fcsv_falbums = 'albums.csv'
albums.to_csv(fcsv_falbums)

In [ ]:
pd.read_csv(fcsv_falbums, index_col=0).head()

## Convert artists data to CSV

In [ ]:
fartists = 'persons.idomaar'

In [ ]:
artists = pd.read_csv(fartists, delimiter='\t', header=None)

In [ ]:
artists.columns = COLUMNS
artists.head()

In [ ]:
artists.set_index('ID', inplace=True)

In [ ]:
artists[artists['Timestamp'] != -1]

In [ ]:
artists[artists['LinkedEntities'] != '{}']

In [ ]:
artists.drop(['Timestamp', 'LinkedEntities'], axis=1, inplace=True)

In [ ]:
print(artists.shape)
artists.head()

In [ ]:
artists_prop = artists['Properties'].apply(lambda s: \
                                           pd.Series({'MBID': json.loads(s)['MBID'], 'Name': json.loads(s)['name']}))

In [ ]:
#set(list(artists.index)) - set(list(artists_prop.index))

In [ ]:
print(artists_prop.shape)
artists_prop.head()

In [ ]:
#artists_prop['Type'] = 'person'
#artists_prop.head()

In [ ]:
#artists.sort_index(inplace=True)

In [ ]:
#artists = artists.merge(artists_prop, left_index=True, right_index=True)
#print(artists.shape)

In [ ]:
artists_prop.loc[297899]

In [ ]:
fcsv_artists = 'persons.csv'
artists_prop.to_csv(fcsv_artists)

In [ ]:
pd.read_csv(fcsv_artists, index_col=0).head()

## Convert users data to CSV

In [ ]:
fusers = 'users.idomaar'

In [ ]:
users = pd.read_csv(fusers, header=None, delimiter='\t')

In [ ]:
users.columns = COLUMNS[:-1]
users.set_index('ID', inplace=True)
users.head()

In [ ]:
def parse_user_properties(props):
    try:
        prop = json.loads(props)
    except:
        props = props.replace('""', 'null').replace(':,', ':null,')
        try: prop = json.loads(props)
        except: print(props); sys.exit(0)
    return pd.Series({'Username': prop['lastfm_username'],
                          'Gender': str.upper(prop['gender']) if prop['gender'] is not None else None,
                          'Age': prop['age'],
                          'Country': prop['country'],
                          'Playcount': prop['playcount'],
                          '#Playlists': prop['playlists'],
                          'Subscribertype': prop['subscribertype']})    

In [ ]:
user_prop = users['Properties'].apply(lambda s: parse_user_properties(s))

In [ ]:
user_prop.shape

In [ ]:
users.drop(['Type', 'Properties'], axis=1, inplace=True)

In [ ]:
users = users.merge(user_prop, left_index=True, right_index=True)
print(users.shape)

In [ ]:
users.head()

In [ ]:
fcsv_users = 'users.csv'
users.to_csv(fcsv_users)

In [ ]:
pd.read_csv(fcsv_users, index_col=0).head()

## Convert tags data to CSV

In [ ]:
ftags = 'tags.idomaar'

In [ ]:
tags = pd.read_csv(ftags, header=None, delimiter='\t')

In [ ]:
tags.columns = COLUMNS
tags.set_index('ID', inplace=True)
tags.head()

In [ ]:
tags[tags['Timestamp'] != -1]

In [ ]:
tags[tags['LinkedEntities'] != '{}']

In [ ]:
tags.drop(['Type', 'Timestamp', 'LinkedEntities'], axis=1, inplace=True)

In [ ]:
print(tags.shape)
tags.head()

In [ ]:
tags.loc[58983, 'Properties']

In [ ]:
def parse_tag_properties(props):
    try:
        prop = json.loads(props)
    except:
        props = props.replace('u"', '').replace('\\', '').replace('\\n', '')\
                     .replace('"value":', 'VALUE:').replace('"url":', 'URL:').replace('"', '')\
                     .replace('VALUE:', '"value":"').replace(', URL:', '", "url":"').replace('}', '"}')
        try: prop = json.loads(props)
        except: print(props); sys.exit(0)
    return pd.Series({'Value': prop['value'].replace('\n', ''), 'URL': prop['url']})

In [ ]:
tags_prop = tags['Properties'].apply(lambda s: parse_tag_properties(s))

In [ ]:
print(tags_prop.shape)
tags_prop.head()

In [ ]:
tags_prop.loc[230795, 'Value']

In [ ]:
fcsv_tags = 'tags.csv'
tags_prop.to_csv(fcsv_tags)

In [ ]:
pd.read_csv(fcsv_tags, index_col=0).head()

## Convert tracks data to CSV

**NOTE**: there are *duplicated lines* (duplicated track ID with possibly different information) in tracks data, need to deal with this.

In [6]:
ftracks = 'tracks.idomaar'

In [7]:
tracks = pd.read_csv(ftracks, header=None, delimiter='\t')

In [8]:
tracks.columns = COLUMNS
#tracks.set_index('ID', inplace=True) # there's duplications
tracks.head()

,Type,ID,Timestamp,Properties,LinkedEntities
0,track,0,-1,"{""duration"":-1,""playcount"":4,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":0}],""albums""..."
1,track,1,-1,"{""duration"":-1,""playcount"":495,""MBID"":null,""na...","{""artists"":[{""type"":""person"",""id"":1}],""albums""..."
2,track,2,-1,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":2}],""albums""..."
3,track,3,-1,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":3}],""albums""..."
4,track,4,-1,"{""duration"":-1,""playcount"":1,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":4}],""albums""..."


In [9]:
tracks[tracks['Timestamp'] != -1]

,Type,ID,Timestamp,Properties,LinkedEntities


In [10]:
tracks.drop(['Type', 'Timestamp'], axis=1, inplace=True)

Check duplications.

In [11]:
tracks['ID'].unique().shape

(4519105,)

In [15]:
print(tracks.shape)

(5675143, 3)


In [16]:
tracks[tracks['ID'] == 170]

,ID,Properties,LinkedEntities
170,170,"{""duration"":212000,""playcount"":2,""MBID"":""d6b90...","{""artists"":[{""type"":""person"",""id"":78}],""albums..."
144980,170,"{""duration"":212000,""playcount"":7826,""MBID"":""d6...","{""artists"":[{""type"":""person"",""id"":78}],""albums..."


In [17]:
#tracks_prop = tracks['Properties'].apply(lambda s: parse_track_properties(s))

In [46]:
prop_columns = ['Duration', 'Playcount', 'MBID', 'Name']
entity_columns = ['ArtistsID', 'AlbumsID', 'TagsID']

In [47]:
tracks_df = pd.DataFrame(columns=['ID'] + prop_columns + entity_columns)
tracks_df.set_index('ID', inplace=True)

In [48]:
tracks.loc[0, 'Properties']

'{"duration":-1,"playcount":4,"MBID":null,"name":"000003+Music+Instructor/_/Dj%27s+Rock+Da+House+%C3%82%E2%89%88%C3%86%E2%89%88%C3%8A01+-+Dj+Max-Pulemet+Vs.+Bomfunk+Mc%27s+-+Electro+Breakdance+party+1+%5B2000%5D+=+CD+ONE%C3%82%E2%89%88%C3%86%E2%89%88%C3%8A"}'

In [49]:
def parse_track_properties(props):
    try:
        prop = json.loads(props)
    except:
        try: prop = json.loads(props)
        except: print(props); sys.exit(0)
    return pd.Series({'Duration': prop['duration'], 'Playcount': prop['playcount'], \
                      'MBID': prop['MBID'], 'Name': prop['name']})

In [50]:
json.loads(tracks.loc[0, 'LinkedEntities'])

{'albums': [], 'artists': [{'id': 0, 'type': 'person'}], 'tags': []}

In [51]:
tracks.loc[0, 'LinkedEntities']

'{"artists":[{"type":"person","id":0}],"albums":[],"tags":[]}'

In [52]:
def parse_track_entities(entities):
    try:
        entity = json.loads(entities)
    except:
        try: entity = json.loads(entities)
        except: print(entities); sys.exit(0)
    return pd.Series({
        'ArtistsID': ','.join([str(x['id']) for x in entity['artists']]) if len(entity['artists']) > 0 else None,
        'AlbumsID': ','.join([str(x['id']) for x in entity['albums']]) if len(entity['albums']) > 0 else None,
        'TagsID': ','.join([str(x['id']) for x in entity['tags']]) if len(entity['tags']) > 0 else None})

In [53]:
parse_track_entities(tracks.loc[0, 'LinkedEntities'])

AlbumsID     None
ArtistsID       0
TagsID       None
dtype: object

In [54]:
def print_progress(cnt, total):
    """Display a progress bar"""
    assert(cnt > 0 and total > 0 and cnt <= total)
    length = 80
    ratio = cnt / total
    n = int(length * ratio)
    sys.stdout.write('\r[%-80s] %d%%' % ('-'*n, int(ratio*100)))
    sys.stdout.flush()

In [55]:
tracks.sort_values(by='ID', axis=0, ascending=True, inplace=True)

In [56]:
tracks.head()

,ID,Properties,LinkedEntities
0,0,"{""duration"":-1,""playcount"":4,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":0}],""albums""..."
1,1,"{""duration"":-1,""playcount"":495,""MBID"":null,""na...","{""artists"":[{""type"":""person"",""id"":1}],""albums""..."
2,2,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":2}],""albums""..."
3,3,"{""duration"":-1,""playcount"":2,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":3}],""albums""..."
4,4,"{""duration"":-1,""playcount"":1,""MBID"":null,""name...","{""artists"":[{""type"":""person"",""id"":4}],""albums""..."


In [ ]:
prev_id = 0
for ix in tracks.index:
    track_id = tracks.loc[ix, 'ID']
    props = parse_track_properties(tracks.loc[ix, 'Properties'])
    entities = parse_track_entities(str(tracks.loc[ix, 'LinkedEntities']))
    if prev_id > 0 and track_id == prev_id:
        # update existing track
        # ['Duration', 'Playcount', 'MBID', 'Name']
        if props['Duration'] is not None and tracks_df.loc[track_id, 'Duration'] is not None and \
           props['Duration'] > tracks_df.loc[track_id, 'Duration']: 
            tracks_df.loc[track_id, 'Duration'] = props['Duration']
        if props['Playcount'] is not None and tracks_df.loc[track_id, 'Playcount'] is not None and \
           props['Playcount'] > tracks_df.loc[track_id, 'Playcount']:
            tracks_df.loc[track_id, 'Playcount'] = props['Playcount']
        if props['MBID'] is not None and tracks_df.loc[track_id, 'MBID'] is not None and \
           len(props['MBID']) > len(tracks_df.loc[track_id, 'MBID']):
            tracks_df.loc[track_id, 'MBID'] = props['MBID']
        if props['Name'] is not None and tracks_df.loc[track_id, 'Name'] is not None and \
           len(props['Name']) > len(tracks_df.loc[track_id, 'Name']):
            tracks_df.loc[track_id, 'Name'] = props['Name']
        # ['ArtistsID', 'AlbumsID', 'TagsID']
        if entities['ArtistsID'] is not None and tracks_df.loc[track_id, 'ArtistsID'] is not None and \
           len(entities['ArtistsID']) > len(tracks_df.loc[track_id, 'ArtistsID']):
            tracks_df.loc[track_id, 'ArtistsID'] = entities['ArtistsID']
        if entities['AlbumsID'] is not None and tracks_df.loc[track_id, 'AlbumsID'] is not None and \
           len(entities['AlbumsID']) > len(tracks_df.loc[track_id, 'AlbumsID']):
            tracks_df.loc[track_id, 'AlbumsID'] = entities['AlbumsID']
        if entities['TagsID'] is not None and tracks_df.loc[track_id, 'TagsID'] is not None and \
           len(entities['TagsID']) > len(tracks_df.loc[track_id, 'TagsID']):
            tracks_df.loc[track_id, 'TagsID'] = entities['TagsID']
    else:
        tracks_df.ID = track_id
        tracks_df.loc[track_id, prop_columns] = props
        tracks_df.loc[track_id, entity_columns] = entities
        prev_id = track_id    
    if (ix+1) % 1000 == 0:
        sys.stdout.write('\r%d / %d' % (ix+1, tracks.shape[0]))
        sys.stdout.flush()

75000

In [ ]:
tracks_df.head()

In [ ]:
fcsv_tracks = 'tracks.csv'
tracks_df.to_csv(fcsv_tracks)

In [ ]:
pd.read_csv(fcsv_tracks, index_col=0).head()

## Convert playlist data to CSV

## Convert user preference data to CSV